In [3]:
!pip install lightgbm

In [2]:
# Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Model
import lightgbm as lgb
#import shap
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

# Configuration
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('train.csv', parse_dates=['date'],index_col=['date'])

In [4]:
df

,store,item,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11
2013-01-03,1,1,14
2013-01-04,1,1,13
2013-01-05,1,1,10
...,...,...,...
2017-12-27,10,50,63
2017-12-28,10,50,59
2017-12-29,10,50,74


In [5]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofweek'] = df.index.dayofweek

In [6]:
df

,store,item,sales,day,month,year,dayofweek
date,,,,,,,
2013-01-01,1,1,13,1,1,2013,1
2013-01-02,1,1,11,2,1,2013,2
2013-01-03,1,1,14,3,1,2013,3
2013-01-04,1,1,13,4,1,2013,4
2013-01-05,1,1,10,5,1,2013,5
...,...,...,...,...,...,...,...
2017-12-27,10,50,63,27,12,2017,2
2017-12-28,10,50,59,28,12,2017,3
2017-12-29,10,50,74,29,12,2017,4


In [7]:
df_test = pd.read_csv('test.csv',parse_dates=['date'],index_col=['date'])
df_test

,id,store,item
date,,,
2018-01-01,0,1,1
2018-01-02,1,1,1
2018-01-03,2,1,1
2018-01-04,3,1,1
2018-01-05,4,1,1
...,...,...,...
2018-03-27,44995,10,50
2018-03-28,44996,10,50
2018-03-29,44997,10,50


In [8]:
x_train = df.loc[df['year'] < 2017].drop(['sales','day', 'dayofweek'], axis=1)
y_train = df.loc[df['year'] < 2017, 'sales']

x_test = df.loc[df['year'] >= 2017].drop(['sales','day', 'dayofweek'], axis=1).reset_index(drop=True)
y_test = df.loc[df['year'] >= 2017, 'sales'].reset_index(drop=True)

In [9]:
first_model = lgb.LGBMRegressor(random_state=384).fit(x_train, y_train, 
                                                      eval_metric= lambda y_true, y_pred: [lgbm_smape(y_true, y_pred)])

In [10]:
#SMAPE: Symmetric mean absolute percentage error (adjusted MAPE)
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

def lgbm_smape(y_true, y_pred):
    smape_val = smape(y_true, y_pred)
    return 'SMAPE', smape_val, False
print("TRAIN SMAPE:", smape(y_train, first_model.predict(x_train)))
print("TEST SMAPE:", smape(y_test, first_model.predict(x_test)))

TRAIN SMAPE: 17.35845673840087
TEST SMAPE: 16.70550443573148


In [11]:
x_test

,store,item,month,year
0,1,1,1,2017
1,1,1,1,2017
2,1,1,1,2017
3,1,1,1,2017
4,1,1,1,2017
...,...,...,...,...
182495,10,50,12,2017
182496,10,50,12,2017
182497,10,50,12,2017
182498,10,50,12,2017


In [14]:
preds = first_model.predict(x_test)

In [15]:
def mean_absolute_percentage_error(y_true, y_pred): 

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [16]:
 mean_absolute_percentage_error(y_test,preds)

17.255051426783382

In [1]:
from sklearn.metrics import mean_squared_error

ms = np.sqrt(mean_squared_error(y_test, preds, squared=False))
ms

D:\Anaconda\envs\demandforecast\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda\envs\demandforecast\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda\envs\demandforecast\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


NameError: name 'np' is not defined

In [18]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,preds)
r2

0.8505245599047762

In [20]:
from sklearn.metrics import mean_absolute_error

MAPE_4_LGBM = mean_absolute_error(y_test, preds)
MAPE_4_LGBM

9.065754721981177

In [22]:
import pickle
## open a file , where you want to store the data
file = open('LGBM.pkl','wb')

##dump information to that file
pickle.dump(first_model, file)